In [80]:
import dotenv
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
import os
from langchain_core.tools import tool
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent,AgentExecutor
from langchain.prompts import (ChatPromptTemplate, AIMessagePromptTemplate, 
                            HumanMessagePromptTemplate, SystemMessagePromptTemplate,MessagesPlaceholder)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [1]:
filePath="D:/MLOPS/Azure_Doc_Intelligence/Samples/layout.pdf"

In [5]:
dotenv.load_dotenv(dotenv_path=dotenv.find_dotenv(filename="../../.env"))

True

In [7]:
endpoint=os.environ["DOC_AI_SERVICE_ENDPOINT_FREE"]
apiKey=os.environ["DOC_AI_API_KEY_FREE"]

In [24]:
loader=AzureAIDocumentIntelligenceLoader(api_endpoint=endpoint,api_key=apiKey,file_path=filePath,mode="page")

In [25]:
documents=loader.load()

In [44]:
type(documents[0].page_content)

str

In [27]:
documents[0].metadata

{'page': 1}

In [69]:
@tool
def content_in_markdown(filePath:str):
    """
        Retrieve the content of a File in a Markdown Format by using the Azure Doc Intelligence API Service
    """
    loader=AzureAIDocumentIntelligenceLoader(api_endpoint=endpoint,api_key=apiKey,file_path=filePath)
    documents=loader.load()
    totalContents=[]
    for document in documents:
        totalContents.append(document.page_content)
    return "\n".join(totalContents)

In [86]:
@tool
def content_in_pageMode(filePath:str):
    """
        Retrieve the content of a File in a non-markdown page-mode Retrieve the content of a File in a Markdown Format by using the Azure Doc Intelligence API Service
    """
    loader=AzureAIDocumentIntelligenceLoader(api_endpoint=endpoint,api_key=apiKey,file_path=filePath,mode='page')
    documents=loader.load()
    totalContents=[]
    for document in documents:
        totalContents.append(document.page_content)
    return "\n".join(totalContents)

In [109]:
tools=[content_in_markdown,content_in_pageMode]

In [110]:
llmOpenAI=ChatOpenAI(max_completion_tokens=4000,temperature=1)

In [111]:
llmOpenAI_with_tools=llmOpenAI.bind_tools(tools=tools)

In [112]:
prompts=ChatPromptTemplate.from_messages(messages=[
        SystemMessagePromptTemplate.from_template(template="""
        You are an intelligent agent who can achieve the goal by accessing to certain agents at your disposal. 
        **Unless explicitly asked to open a document through the Markdown mode, always open it through `content_in_pageMode` agent**
        """),
        MessagesPlaceholder(variable_name="chat_history",optional=True),
        HumanMessagePromptTemplate.from_template(template="{user_input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [113]:
agent=create_tool_calling_agent(llm=llmOpenAI, tools=tools,prompt=prompts)

In [114]:
agentExecutor=AgentExecutor(tools=tools,agent=agent,verbose=True)

In [115]:
messageHistory=ChatMessageHistory()

In [116]:
agentExecutorWithMemory=RunnableWithMessageHistory(runnable=agentExecutor,
                                                   get_session_history=lambda sess_id: messageHistory,
                                                   input_messages_key="user_input",
                                                   history_messages_key="chat_history"
                                                  )

In [117]:
agentExecutorWithMemory.invoke(
        input={"user_input":"Can you open the pdf located in the directory path: `samples/layout.pdf` and summarize the same? "},
        config={"configurable":{"session_id":"<trial_1>"}}
    )



> Entering new AgentExecutor chain...

Invoking: `content_in_pageMode` with `{'filePath': 'samples/layout.pdf'}`


UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-Q ☐ ☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Quarterly Period Ended March 31, 2020 OR TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Transition Period From to Commission File Number 001-37845 MICROSOFT CORPORATION WASHINGTON (STATE OF INCORPORATION) ONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399 (425) 882-8080 www.microsoft.com/investor 91-1144442 (I.R.S. ID) Securities registered pursuant to Section 12(b) of the Act: Title of each class Trading Symbol Name of exchange on which registered Common stock, $0.00000625 par value per share MSFT NASDAQ 2.125% Notes due 2021 MSFT NASDAQ 3.125% Notes due 2028 MSFT NASDAQ 2.625% Notes due 2033 MSFT NASDAQ Securities registered pursuant to Secti

{'user_input': 'Can you open the pdf located in the directory path: `samples/layout.pdf` and summarize the same? ',
 'chat_history': [],
 'output': "The document is a Form 10-Q filed with the United States Securities and Exchange Commission (SEC) by Microsoft Corporation. It contains information about Microsoft Corporation, including its address, securities registered, and various checkmark indicators related to filings and compliance.\n\nHere are some key points from the document:\n- Company Name: Microsoft Corporation\n- Address: One Microsoft Way, Redmond, Washington 98052-6399\n- Trading Symbol: MSFT\n- Securities Registered: Common Stock, 2.125% Notes due 2021, 3.125% Notes due 2028, 2.625% Notes due 2033\n- Number of Shares Outstanding: 7,583,440,247 shares as of April 24, 2020\n\nThe document provides details about the company's filings, compliance status, and other relevant information required for regulatory reporting."}

In [118]:
agentExecutorWithMemory.invoke(
        input={"user_input":"What has been mentioned about the Securities and Exchange Commission? "},
        config={"configurable":{"session_id":"<trial_1>"}}
    )



> Entering new AgentExecutor chain...

Invoking: `content_in_pageMode` with `{'filePath': 'samples/layout.pdf'}`


UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-Q ☐ ☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Quarterly Period Ended March 31, 2020 OR TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Transition Period From to Commission File Number 001-37845 MICROSOFT CORPORATION WASHINGTON (STATE OF INCORPORATION) ONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399 (425) 882-8080 www.microsoft.com/investor 91-1144442 (I.R.S. ID) Securities registered pursuant to Section 12(b) of the Act: Title of each class Trading Symbol Name of exchange on which registered Common stock, $0.00000625 par value per share MSFT NASDAQ 2.125% Notes due 2021 MSFT NASDAQ 3.125% Notes due 2028 MSFT NASDAQ 2.625% Notes due 2033 MSFT NASDAQ Securities registered pursuant to Secti

{'user_input': 'What has been mentioned about the Securities and Exchange Commission? ',
 'chat_history': [HumanMessage(content='Can you open the pdf located in the directory path: `samples/layout.pdf` and summarize the same? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The document is a Form 10-Q filed with the United States Securities and Exchange Commission (SEC) by Microsoft Corporation. It contains information about Microsoft Corporation, including its address, securities registered, and various checkmark indicators related to filings and compliance.\n\nHere are some key points from the document:\n- Company Name: Microsoft Corporation\n- Address: One Microsoft Way, Redmond, Washington 98052-6399\n- Trading Symbol: MSFT\n- Securities Registered: Common Stock, 2.125% Notes due 2021, 3.125% Notes due 2028, 2.625% Notes due 2033\n- Number of Shares Outstanding: 7,583,440,247 shares as of April 24, 2020\n\nThe document provides details about the company's filing

In [119]:
agentExecutorWithMemory.invoke(
        input={"user_input":"Can you open the pdf located in the directory path: `samples/layout.pdf` and summarize the same through Markdown? "},
        config={"configurable":{"session_id":"<trial_2>"}}
    )



> Entering new AgentExecutor chain...

Invoking: `content_in_markdown` with `{'filePath': 'samples/layout.pdf'}`


# UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549


## FORM 10-Q

☐
☒
☒
QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the Quarterly Period Ended March 31, 2020
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the Transition Period From
to

Commission File Number 001-37845


## MICROSOFT CORPORATION

WASHINGTON
(STATE OF INCORPORATION)
ONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399
(425) 882-8080
www.microsoft.com/investor

91-1144442
(I.R.S. ID)

Securities registered pursuant to Section 12(b) of the Act:


<table>
<tr>
<th>Title of each class</th>
<th>Trading Symbol</th>
<th>Name of exchange on which registered</th>
</tr>
<tr>
<td>Common stock, $0.00000625 par value per share</td>
<td>MSFT</td>
<td>NASDAQ</td>
</tr>
<tr>
<td>2.125% Notes due 2021</td

{'user_input': 'Can you open the pdf located in the directory path: `samples/layout.pdf` and summarize the same through Markdown? ',
 'chat_history': [HumanMessage(content='Can you open the pdf located in the directory path: `samples/layout.pdf` and summarize the same? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The document is a Form 10-Q filed with the United States Securities and Exchange Commission (SEC) by Microsoft Corporation. It contains information about Microsoft Corporation, including its address, securities registered, and various checkmark indicators related to filings and compliance.\n\nHere are some key points from the document:\n- Company Name: Microsoft Corporation\n- Address: One Microsoft Way, Redmond, Washington 98052-6399\n- Trading Symbol: MSFT\n- Securities Registered: Common Stock, 2.125% Notes due 2021, 3.125% Notes due 2028, 2.625% Notes due 2033\n- Number of Shares Outstanding: 7,583,440,247 shares as of April 24, 2020\n\nThe document

In [120]:
agentExecutorWithMemory.invoke(
        input={"user_input":"What has been mentioned about the Securities and Exchange Commission? "},
        config={"configurable":{"session_id":"<trial_2>"}}
    )



> Entering new AgentExecutor chain...

Invoking: `content_in_markdown` with `{'filePath': 'samples/layout.pdf'}`


# UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549


## FORM 10-Q

☐
☒
☒
QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the Quarterly Period Ended March 31, 2020
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the Transition Period From
to

Commission File Number 001-37845


## MICROSOFT CORPORATION

WASHINGTON
(STATE OF INCORPORATION)
ONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399
(425) 882-8080
www.microsoft.com/investor

91-1144442
(I.R.S. ID)

Securities registered pursuant to Section 12(b) of the Act:


<table>
<tr>
<th>Title of each class</th>
<th>Trading Symbol</th>
<th>Name of exchange on which registered</th>
</tr>
<tr>
<td>Common stock, $0.00000625 par value per share</td>
<td>MSFT</td>
<td>NASDAQ</td>
</tr>
<tr>
<td>2.125% Notes due 2021</td

{'user_input': 'What has been mentioned about the Securities and Exchange Commission? ',
 'chat_history': [HumanMessage(content='Can you open the pdf located in the directory path: `samples/layout.pdf` and summarize the same? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The document is a Form 10-Q filed with the United States Securities and Exchange Commission (SEC) by Microsoft Corporation. It contains information about Microsoft Corporation, including its address, securities registered, and various checkmark indicators related to filings and compliance.\n\nHere are some key points from the document:\n- Company Name: Microsoft Corporation\n- Address: One Microsoft Way, Redmond, Washington 98052-6399\n- Trading Symbol: MSFT\n- Securities Registered: Common Stock, 2.125% Notes due 2021, 3.125% Notes due 2028, 2.625% Notes due 2033\n- Number of Shares Outstanding: 7,583,440,247 shares as of April 24, 2020\n\nThe document provides details about the company's filing